In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/02/18 15:51:06 WARN Utils: Your hostname, isolde resolves to a loopback address: 127.0.1.1; using 192.168.0.188 instead (on interface wlp1s0)
23/02/18 15:51:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/18 15:51:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/18 15:51:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [30]:
csv_path = "data/fhvhv_tripdata_2021-01.csv"

df = spark.read \
    .option("header", "true") \
    .csv(csv_path)

In [31]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

In [32]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [38]:
!head -n 1001 data/fhvhv_tripdata_2021-01.csv > data/head.csv

In [39]:
!wc -l data/head.csv

1001 data/head.csv


In [40]:
import pandas as pd

In [41]:
df_pandas = pd.read_csv("data/head.csv")

In [42]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [45]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [46]:
from pyspark.sql import types

In [47]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [48]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('data/fhvhv_tripdata_2021-01.csv')

In [50]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [51]:
df = df.repartition(24)

In [56]:
df.write.parquet('data/fhvhv/2021/01/')

23/02/18 16:45:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [57]:
df = spark.read.parquet('data/fhvhv/2021/01/')

In [58]:
df

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [59]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [63]:
df.select("pickup_datetime", "dropoff_datetime", "PULocationID", "DOLocationID") \
  .filter(df.hvfhs_license_num == "HV0003").show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-01 17:05:32|2021-01-01 17:12:27|         122|         191|
|2021-01-03 08:53:52|2021-01-03 08:59:41|          37|          36|
|2021-01-05 08:28:47|2021-01-05 09:01:57|         208|          43|
|2021-01-05 09:10:43|2021-01-05 09:32:00|         223|         151|
|2021-01-03 12:58:12|2021-01-03 13:01:33|         243|         244|
|2021-01-01 22:15:45|2021-01-01 22:19:50|         223|         223|
|2021-01-04 06:40:59|2021-01-04 06:47:16|          39|          91|
|2021-01-04 17:43:16|2021-01-04 17:49:46|         174|          20|
|2021-01-04 23:20:01|2021-01-04 23:29:36|          50|          90|
|2021-01-02 10:22:24|2021-01-02 10:46:19|         148|         151|
|2021-01-03 17:30:47|2021-01-03 17:44:59|         127|         169|
|2021-01-02 00:55:28|2021-01-02 01:04:53|       

In [64]:
from pyspark.sql import functions as F

In [66]:
df.withColumn("pickup_date", df.pickup_datetime) \
  .withColumn("dropoff_date", df.dropoff_datetime) \
  .select("pickup_datetime", "dropoff_datetime", "PULocationID", "DOLocationID") \
  .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-01 17:05:32|2021-01-01 17:12:27|         122|         191|
|2021-01-03 08:53:52|2021-01-03 08:59:41|          37|          36|
|2021-01-02 13:17:45|2021-01-02 13:49:14|          48|         265|
|2021-01-05 08:28:47|2021-01-05 09:01:57|         208|          43|
|2021-01-05 09:10:43|2021-01-05 09:32:00|         223|         151|
|2021-01-04 10:35:09|2021-01-04 10:44:37|         252|          51|
|2021-01-03 12:58:12|2021-01-03 13:01:33|         243|         244|
|2021-01-01 22:15:45|2021-01-01 22:19:50|         223|         223|
|2021-01-04 06:40:59|2021-01-04 06:47:16|          39|          91|
|2021-01-04 17:43:16|2021-01-04 17:49:46|         174|          20|
|2021-01-04 23:20:01|2021-01-04 23:29:36|          50|          90|
|2021-01-02 10:22:24|2021-01-02 10:46:19|       